# OmniBot

In [1]:
import sys
sys.executable

'/Users/panos/Library/Caches/pypoetry/virtualenvs/omnibot-lHmdBll9-py3.9/bin/python'

## Setup

In [2]:
## LOCAL .env file
from dotenv import dotenv_values
secrets = dotenv_values("../.env")
OPENAI_API_KEY = secrets['OPENAI_API_KEY']

## Step 2: Initialize Embeddings engine
from langchain.embeddings.openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

In [3]:
# Load the persisted database from disk
from langchain.vectorstores import Chroma
persist_directory="../db"
vectordb = Chroma(persist_directory=persist_directory, 
                  embedding_function = embeddings)

Using embedded DuckDB with persistence: data will be stored in: ../db


## Retriever

In [4]:
# Make a Retriever
retriever = vectordb.as_retriever(search_kwargs={'k':2})

In [5]:
retriever.search_type

'similarity'

In [6]:
docs = retriever.get_relevant_documents("Τι είναι το ίδρυμα Αθανάσιος Κ. Λασκαρίδης?")

In [7]:
len(docs)

2

In [7]:
docs

[Document(page_content='Ίδρυμα Αθανάσιος Κ. Λασκαρίδης υποστηρίζει σε σταθερή βάση το φιλανθρωπικό σωματείο Άγιος Πολύκαρπος Σμύρνης, το οποίο περιθάλπει και υποστηρίζει παιδιά που έχουν ανάγκη από δωρεάν στέγαση, διατροφή, εκπαίδευση, διαπαιδαγώγηση και ευκαιρίες κοινωνικής αποκατάστασης. Κουλούρειο Νοσοκομείο Ύδρας Παναγία Φανερωμένη Το Κοινωφελές Ίδρυμα Αθανάσιος Κ. Λασκαρίδηςυποστηρίζει τη λειτουργία του Κουλούρειου', metadata={'source': '../data/93.txt'}),
 Document(page_content='https://www.aclcf.org/alla-programmata/politistiki-klironomia/ | ΠΟΛΙΤΙΣΤΙΚΗ ΚΛΗΡΟΝΟΜΙΑ Μέσα από ένα ευρύ φάσμα δράσεων και συνεργειών, το Κοινωφελές Ίδρυμα Αθανάσιος Κ. Λασκαρίδης αναδεικνύει τον πολιτισμό ως μία από τις προτεραιότητές του. Προστασία και ανάδειξη μνημείων και επιλεκτικές παρεμβάσεις στον δημόσιο χώρο αποτελούν τον βασικό κορμό του σχεδιασμού του Ιδρύματος για την διατήρηση της', metadata={'source': '../data/103.txt'})]

In [8]:
docs = retriever.get_relevant_documents("Ποιες είναι οι 3 τελευταίες δράσεις του ιδρύματος?")

In [9]:
docs

[Document(page_content='ΤΩΝ ΜΑΘΗΤΩΝ/ΤΡΙΩΝ ΩΣ ΕΝΕΡΓΩΝ ΠΟΛΙΤΩΝ', metadata={'source': '../data/75.txt'}),
 Document(page_content='ΠΡΟΓΡΑΜΜΑ ΓΙΑ ΤΗ ΔΙΑΜΟΡΦΩΣΗ ΤΩΝ ΜΑΘΗΤΩΝ/ΤΡΙΩΝ ΩΣ ΕΝΕΡΓΩΝ ΠΟΛΙΤΩΝ', metadata={'source': '../data/97.txt'})]

## Retrieval QA

In [71]:
from langchain.prompts import PromptTemplate
prompt_template = """You are OmniBot, a chatbot responsible for answering questions about the Athanasios K. Laskaridis Foundation.

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.
Reply with a single sentance and always finish your answer with a period.

{context}

Question: {question}

Helpful Answer:"
"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

In [83]:
# Create the chain to answer the question
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI

chain_type_kwargs = {"prompt": PROMPT}

qa_chain = RetrievalQA.from_chain_type(llm=OpenAI(temperature=0, max_tokens=2000),
                                       chain_type="stuff",
                                       retriever=retriever,
                                       return_source_documents=False,
                                       chain_type_kwargs=chain_type_kwargs)

In [84]:
print(qa_chain.combine_documents_chain.llm_chain.prompt.template)

You are OmniBot, a chatbot responsible for answering questions about the Athanasios K. Laskaridis Foundation.

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.
Reply with a single sentance and always finish your answer with a period.

{context}

Question: {question}

Helpful Answer:"



In [80]:
# Q
query = "Hello, who are you?"
qa_chain({'query': query})

{'query': 'Hello, who are you?',
 'result': 'I am OmniBot, a chatbot responsible for answering questions about the Athanasios K. Laskaridis Foundation.'}

In [85]:
# Q
query = "Τι είναι το ίδρυμα Αθανάσιος Κ. Λασκαρίδης?"
qa_chain({'query': query})


{'query': 'Τι είναι το ίδρυμα Αθανάσιος Κ. Λασκαρίδης?',
 'result': 'Το Ίδρυμα Αθανάσιος Κ. Λασκαρίδης είναι ένα κοινωφελές Ίδρυμα που υποστηρίζει την διατήρηση του πολιτισμού μέσω δράσεων και συνεργασιών, όπως την προστασία και ανάδειξη μνημείων και τις επιλεκτικές παρεμβάσεις στον δημόσιο χώρο.'}

In [86]:
# Q
query = "Με ποιους φορείς συνεργάζεται το ίδρυμα?"
qa_chain({'query': query})

{'query': 'Με ποιους φορείς συνεργάζεται το ίδρυμα?',
 'result': 'Το Κοινωφελές Ίδρυμα Αθανάσιος Κ. Λασκαρίδης συνεργάζεται με ένα ευρύ πελατολόγιο το οποίο περιλαμβάνει πολυεθνικές εταιρίες, μικρές επιχειρήσεις, οργανισμούς του ιδιωτικού και του δημοσίου τομέα, καθώς και εκπροσώπους της πολιτικής, της τέχνης και της επιστήμης. Επίσης, έχει διατελέσει επιστημονικός συνεργάτης στη Βουλή των Ελλήνων, σύμβουλος του Υπουργού Πολιτισμού, καθώς και θεματικός συνεργάτης βουλευτών και αιρετών της αυτοδιοίκησης.'}

In [89]:
# Q
query = "Ποιες ομάδες διακρίθηκαν στον Μαραθώνιο καινοτομίας 2022?"
qa_chain({'query': query})

{'query': 'Ποιες ομάδες διακρίθηκαν στον Μαραθώνιο καινοτομίας 2022?',
 'result': 'Οι 5 ομάδες που διακρίθηκαν είναι οι ακόλουθες: Dataphoria, E-Voting, Smart Mobility, Smart Energy και Smart Agriculture.'}